In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

## Data Preprocessing 

In [2]:
# Importing the dataset
application_df = pd.read_csv('Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Removing 'EIN' and 'NAME' columns
application_df.drop(['EIN', 'NAME'], axis='columns', inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Number of unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Deeper look at 'APPLICATION_TYPE' value counts
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Binning 'APPLICATION_TYPE' under 500
# Putting T9-T17 (according to their frequencies) into a single bin called 'Other'
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replacing low frequency application types in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# 'CLASSIFICATION' column value counts
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# 'CLASSIFICATION' column value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Binning 'CLASSIFICATION' under 1000
classifications = application_df['CLASSIFICATION'].unique()
classifications_to_replace = [x for x in classifications if x not in ('C1000', 'C2000', 'C1200', 'C3000', 'C2100')]

# Replacing low frequency application types in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Converting categorical data to numeric
application_num = pd.get_dummies(application_df)
application_num.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Splitting into features and target arrays
y = application_num['IS_SUCCESSFUL'].values
X = application_num.drop('IS_SUCCESSFUL', axis='columns').values

# Splitting into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [12]:
scaler = StandardScaler()

# Fitting the scaler with X_train
scaler.fit(X_train)

# Scaling the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train, and Evaluate the Model

In [13]:
# Model: Deep Neural Network 
# Input features same length as X training dataset, 80 nodes in hidden layer 1, 30 nodes in hidden layer 2, and sigmoid output layer

nn = tf.keras.models.Sequential()

# First hiden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=len(X_train[0])))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Checking structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compiling the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Callback to save the model's weight every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'Checkpoints/weights-{epoch:03d}-{accuracy:.4f}.ckpt',
    period = 5,
    monitor = 'accuracy',
    verbose = 0,
    mode = 'auto',
    save_weights_only = True
)

In [18]:
# Training the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7325
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7342
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7334
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7347
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7353
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7356
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7358
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7354
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7350
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accura

In [19]:
# Evaluating model using test data
model_loss, model_accurary = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accurary}')

268/268 - 0s - loss: 0.5753 - accuracy: 0.7237 - 326ms/epoch - 1ms/step
Loss: 0.5752522349357605, Accuracy: 0.7237317562103271


In [20]:
# Exporting model to HDF5 file
nn.save('Model/AlphabetSoupCharity.h5')